In [1]:
import pandas as pd
import numpy as np
data = pd.read_excel("dataWithSomeReformatting.xlsx", index_col = [0])
keys = list(data.keys())
keys

['Article_title',
 'Article_link',
 'General_outcome',
 'General_outcome (1,0)',
 'Desc_general_outcome',
 'Daily_intake',
 'Descr_daily_intake',
 'Health_outcome',
 'Descr_health_outcome',
 'Environmental_outcome',
 'Descr_environmental_outcome',
 'Fund_source',
 'Fund_Source',
 'Author_affiliation',
 'Loc_authors',
 'Loc_experiment',
 'Year',
 'Journal',
 'Loc_journal',
 'Loc_editor_journal',
 'Current Impact_Factor',
 'Impact_F_Publishing_Year',
 'Num_Citations',
 'Public',
 'Private',
 'University',
 'International',
 'Research',
 'MEXICO',
 'ARGENTINA',
 'UKRAINE',
 'LATVIA',
 'GERMANY',
 'NEW ZEALAND',
 'BRAZL',
 'NETHERLANDS',
 'NIGERIA',
 'SWEDEN',
 'FRANCE',
 'PORTUGAL',
 'CROATIA',
 'SLOVENIA',
 'SPAIN',
 'POLAND',
 'ITALY',
 'ECUADOR',
 'MOROCCO',
 'GREECE',
 'INDIA',
 'IRAQ',
 'UK',
 'JAPAN',
 'ROMANIA',
 'CHINA',
 'IRAN',
 'IRELAND',
 'AUSTRIA',
 'TUNISIA',
 'SLOVAK REPUBLIC',
 'FINLAND',
 'TAIWAN',
 'AUSTRALIA',
 'TURKEY',
 'REPUBLIC OF KOREA',
 'SRI LANKA',
 'SOUTH AFRIC

In [2]:
years = keys[-31:]
countries = keys[-82:-31]
data.rename(columns = {year:str(year) for year in years}, inplace = True)
years = [str(year) for year in years]

In [3]:
# drop Iran due to colinearity
#del years[0]
keys =['Current Impact_Factor',
 'Num_Citations',
 'Public',
 'Private',
 'University',
 'International',
 'Research']
#keys = ["General_outcome", "Year", "Fund_Source", "Loc_experiment", "Journal"]
max_cond_vars = len(keys) - 1
dag_keys = keys #+ years #+ countries
data[dag_keys]

,Current Impact_Factor,Num_Citations,Public,Private,University,International,Research
0,2.790,1.0,1,0,0,0,0
1,1.812,29.0,1,0,0,0,0
2,0.680,31.0,0,0,0,0,0
3,2.950,20.0,0,1,0,0,0
4,4.679,108.0,1,0,0,0,0
...,...,...,...,...,...,...,...
498,4.872,256.0,0,0,0,0,0
499,5.715,5.0,1,0,0,0,0
500,3.595,0.0,1,0,0,0,0
501,7.314,42.0,0,0,0,0,0


In [4]:
import pingouin

def gen_pcorr(df, method = "pearson", sig = 0.01):
    # Correlation type:
    # 'pearson': Pearson r product-moment correlation
    # 'spearman': Spearman ρ rank-order correlation
    # 'kendall': Kendall’s τB correlation (for ordinal data)
    # 'bicor': Biweight midcorrelation (robust)
    # 'percbend': Percentage bend correlation (robust)
    # 'shepherd': Shepherd’s pi correlation (robust)
    # 'skipped': Skipped correlation (robust)
    pcs_dct = {}
    sig_corr_dct = {}
    for x in df.keys():
        sig_corr_dct[x] = []
        pcs_dct[x]={}
        for y in df.keys():
            # control variables
            # select variables that are not x or y
            other_vars = [z for z in df.keys() if z != y and z != x ]
            if x == y:
                # No need to calculate if the variable is itself
                pcs_dct[x][y] = 1
            else:
                pcs_dct[x][y] = df.partial_corr(x=x,y=y, covar=other_vars,
                                      method=method).round(3)
                if pcs_dct[x][y]["p-val"].values[0] < sig:
                    sig_corr_dct[x].append((y, pcs_dct[x][y]["r"].values[0]))

    return pcs_dct, sig_corr_dct

In [5]:
pcs_dct, sig_corr_dct = gen_pcorr(data[dag_keys].fillna(0), method = "pearson", sig = 0.1)

In [6]:
sig_corr_dct

{'Current Impact_Factor': [('Num_Citations', 0.216),
  ('Public', 0.194),
  ('Private', 0.108),
  ('University', 0.082),
  ('International', 0.112)],
 'Num_Citations': [('Current Impact_Factor', 0.216), ('Private', 0.126)],
 'Public': [('Current Impact_Factor', 0.194),
  ('Private', -0.166),
  ('International', -0.149)],
 'Private': [('Current Impact_Factor', 0.108),
  ('Num_Citations', 0.126),
  ('Public', -0.166),
  ('University', -0.169),
  ('International', -0.091)],
 'University': [('Current Impact_Factor', 0.082),
  ('Private', -0.169),
  ('International', -0.075)],
 'International': [('Current Impact_Factor', 0.112),
  ('Public', -0.149),
  ('Private', -0.091),
  ('University', -0.075)],
 'Research': []}

In [7]:
pcs_dct["Current Impact_Factor"]["Num_Citations"]

,n,r,CI95%,p-val
pearson,503,0.216,"[0.13, 0.3]",0.0


In [9]:
import statsmodels.api as sm
import numpy

residuals = {}
partial_corr = {}
reg_df = data[dag_keys].dropna()
for y_var in dag_keys:
    # save a list of all variables except for the y_var
    X_vars = [x for x in dag_keys if x != y_var]
#    print(y_var,":", X_vars)
    
    X = reg_df[X_vars]
    X["constant"] = 1
    y = reg_df[[y_var]]
    
    model = sm.OLS(y, X)
    results = model.fit()
    predict = results.predict 
    reg_df[y_var + " predict"] = predict()
    reg_df[y_var+ " residual"] = results.resid
    residuals[y_var] = results.resid
    
residuals

<ipython-input-9-db9f5d0fd763>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["constant"] = 1


{'Current Impact_Factor': 0     -0.142293
 1     -1.251786
 2     -1.769331
 3     -0.173571
 4      1.244213
          ...   
 498    1.366022
 499    2.763922
 500    0.667404
 501    4.813011
 502   -0.646958
 Length: 491, dtype: float64,
 'Num_Citations': 0      -37.726567
 1       -0.771373
 2       13.693491
 3      -50.238300
 4       51.976540
           ...    
 498    200.308856
 499    -60.509739
 500    -46.097662
 501    -36.051659
 502    -34.499595
 Length: 491, dtype: float64,
 'Public': 0      0.335669
 1      0.379871
 2     -0.566471
 3     -0.391460
 4      0.237074
          ...   
 498   -0.784248
 499    0.196269
 500    0.297477
 501   -0.882793
 502    0.395827
 Length: 491, dtype: float64,
 'Private': 0     -0.065368
 1     -0.058215
 2     -0.142103
 3      0.815650
 4     -0.155694
          ...   
 498   -0.336637
 499   -0.128437
 500   -0.081728
 501   -0.286061
 502    0.052461
 Length: 491, dtype: float64,
 'University': 0     -0.234017
 1     -0.219266

In [11]:
for x in dag_keys:
    partial_corr[x] = {}
    for y in dag_keys:
        partial_corr[x][y] = {}
        
        Y = pd.DataFrame(residuals[y])
        X = pd.DataFrame(residuals[x])
        model = sm.OLS(Y,X)
        results = model.fit()
#        print(results.summary())
#        print("R2:", results.rsquared, 
#              "P-value:", results.pvalues)
        if x == y:
            partial_corr[x][y]["r"] = 1
        else:
            df = pd.DataFrame([residuals[x], residuals[y]])
            partial_corr[x][y]["r"] = df.T.corr()[0][1] * -1
        #[0][1] * -1
        partial_corr[x][y]["p-val"] = results.pvalues[0]
        partial_corr[x][y]["summary"] = results.summary()
print(partial_corr["Current Impact_Factor"]["International"])
# .sort() command is alphabetizing index and column
#pd.DataFrame(partial_corr).sort_index(axis=0, ascending=True).sort_index(axis=1, ascending = True) 
partial_corr

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1804: RuntimeWarning: divide by zero encountered in double_scalars
  return self.mse_model/self.mse_resid
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:903: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:50: RuntimeWarning: invalid value encountered in double_scalars
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1804: RuntimeWarning: divide by zero encountered in double_scalars
  return self.mse_model/self.mse_resid
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:903: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
C:\Progr

{'r': 0.1308130830838091, 'p-val': 0.0036527070512031327, 'summary': <class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
Dep. Variable:                      0   R-squared (uncentered):                   0.017
Model:                            OLS   Adj. R-squared (uncentered):              0.015
Method:                 Least Squares   F-statistic:                              8.531
Date:                Tue, 10 Aug 2021   Prob (F-statistic):                     0.00365
Time:                        08:16:46   Log-Likelihood:                          255.17
No. Observations:                 491   AIC:                                     -508.3
Df Residuals:                     490   BIC:                                     -504.1
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
   

{'Current Impact_Factor': {'Current Impact_Factor': {'r': 1,
   'p-val': 0.0,
   'summary': <class 'statsmodels.iolib.summary.Summary'>
   """
                                    OLS Regression Results                                
   Dep. Variable:                      0   R-squared (uncentered):                   1.000
   Model:                            OLS   Adj. R-squared (uncentered):              1.000
   Method:                 Least Squares   F-statistic:                          1.806e+34
   Date:                Tue, 10 Aug 2021   Prob (F-statistic):                        0.00
   Time:                        08:16:46   Log-Likelihood:                          16890.
   No. Observations:                 491   AIC:                                 -3.378e+04
   Df Residuals:                     490   BIC:                                 -3.377e+04
   Df Model:                           1                                                  
   Covariance Type:            nonrobu

In [18]:
reg_df[dag_keys].pcorr().sort_index(axis=0, ascending=True).sort_index(axis=1, ascending = True) 

,Current Impact_Factor,International,Num_Citations,Private,Public,Research,University
Current Impact_Factor,1.000000,0.130813,0.207368,0.123208,0.172225,0.030213,0.076942
International,0.130813,1.000000,-0.014726,-0.092324,-0.142764,-0.026761,-0.072335
Num_Citations,0.207368,-0.014726,1.000000,0.123566,0.013126,0.018961,0.023583
Private,0.123208,-0.092324,0.123566,1.000000,-0.173402,-0.048535,-0.169066
Public,0.172225,-0.142764,0.013126,-0.173402,1.000000,-0.061226,-0.046923
Research,0.030213,-0.026761,0.018961,-0.048535,-0.061226,1.000000,-0.053023
University,0.076942,-0.072335,0.023583,-0.169066,-0.046923,-0.053023,1.000000
